In [1]:
# !jupyter lab build

In [2]:
import sys
from pathlib import Path

# Add src directory to Python path
project_root = Path.cwd().parent.parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"Added {src_path} to Python path")

Added /Users/cooper/Desktop/hydro-forecasting/src to Python path


In [3]:
from sklearn.pipeline import Pipeline

from hydro_forecasting.data.caravanify_parquet import (
    CaravanifyParquet,
    CaravanifyParquetConfig,
)
from hydro_forecasting.experiment_utils.hyperparameter_tune_model import hyperparameter_tune_model
from hydro_forecasting.preprocessing.grouped import GroupedPipeline
from hydro_forecasting.preprocessing.normalize import NormalizeTransformer
from hydro_forecasting.preprocessing.standard_scale import StandardScaleTransformer

---

## Experiment constants

In [ ]:
REGIONS = ["CA"]

COUNTRY = "tajikistan"

## Loading the data (as gauge_ids)

In [ ]:
def load_basin_ids(country: str) -> list[str]:
    """
    Function to load basins for a given country in Central Asia
    """
    # Make country lowercase and make the first letter uppercase
    country = country.lower()
    country = country.capitalize()

    if country != "Tajikistan" and country != "Kyrgyzstan":
        print("Country not supported")
        return []

    configs = CaravanifyParquetConfig(
        attributes_dir="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes",
        timeseries_dir="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/timeseries/csv",
        gauge_id_prefix="CA",
        use_hydroatlas_attributes=True,
        use_caravan_attributes=True,
        use_other_attributes=True,
    )

    caravan = CaravanifyParquet(configs)
    ca_basins = caravan.get_all_gauge_ids()
    caravan.load_stations(ca_basins)
    static_data = caravan.get_static_attributes()

    return list(static_data[static_data["country"] == country]["gauge_id"].unique())


basin_ids = load_basin_ids(COUNTRY)

## Datamodule Configs

In [ ]:
region_time_series_base_dirs = {
    region: f"/Users/cooper/Desktop/CaravanifyParquet/{region}/post_processed/timeseries/csv/{region}"
    for region in REGIONS
}

region_static_attributes_base_dirs = {
    region: f"/Users/cooper/Desktop/CaravanifyParquet/{region}/post_processed/attributes/{region}" for region in REGIONS
}

path_to_preprocessing_output_directory = f"/Users/cooper/Desktop/hydro-forecasting/experiments/HPT/data_cache/{COUNTRY}"

In [7]:
forcing_features = [
    "snow_depth_water_equivalent_mean",
    "surface_net_solar_radiation_mean",
    "surface_net_thermal_radiation_mean",
    "potential_evaporation_sum_ERA5_LAND",
    "potential_evaporation_sum_FAO_PENMAN_MONTEITH",
    "temperature_2m_mean",
    "temperature_2m_min",
    "temperature_2m_max",
    "total_precipitation_sum",
]

static_features = [
    "p_mean",
    "area",
    "ele_mt_sav",
    "high_prec_dur",
    "frac_snow",
    "high_prec_freq",
    "slp_dg_sav",
    "cly_pc_sav",
    "aridity_ERA5_LAND",
    "aridity_FAO_PM",
]

target = "streamflow"

In [8]:
feature_pipeline = GroupedPipeline(
    Pipeline([("scaler", StandardScaleTransformer()), ("normalizer", NormalizeTransformer())]),
    columns=forcing_features,
    group_identifier="gauge_id",
)

target_pipeline = GroupedPipeline(
    Pipeline([("scaler", StandardScaleTransformer()), ("normalizer", NormalizeTransformer())]),
    columns=["streamflow"],
    group_identifier="gauge_id",
)

static_pipeline = Pipeline([("scaler", StandardScaleTransformer())])

preprocessing_config = {
    "features": {"pipeline": feature_pipeline},
    "target": {"pipeline": target_pipeline},
    "static_features": {"pipeline": static_pipeline, "columns": static_features},
}

In [9]:
datamodule_config = {
    "output_length": 10,
    "region_time_series_base_dirs": region_time_series_base_dirs,
    "region_static_attributes_base_dirs": region_static_attributes_base_dirs,
    "path_to_preprocessing_output_directory": path_to_preprocessing_output_directory,
    "group_identifier": "gauge_id",
    "batch_size": 2048,
    "forcing_features": forcing_features,
    "static_features": static_features,
    "target": target,
    "num_workers": 4,
    "min_train_years": 5,
    "train_prop": 0.5,
    "val_prop": 0.25,
    "test_prop": 0.25,
    "max_imputation_gap_size": 5,
    "chunk_size": 100,
    "validation_chunk_size": 100,
    "is_autoregressive": True,
    "preprocessing_configs": preprocessing_config,
}

## Training Configs

In [ ]:
training_config = {
    "max_epochs": 200,
    "accelerator": "mps",
    "devices": 1,
    "early_stopping_patience": 15,
    "reload_dataloaders_every_n_epochs": False,
    "enable_progress_bar": True,
}

## Remaining Configs

In [11]:
output_dir = "/Users/cooper/Desktop/hydro-forecasting/experiments/HPT"
model_type = "tide"
experiment_name = f"HPT_{model_type}_{COUNTRY}"
num_trials = 5

## Training the models from scratch

In [12]:
hpt_study = hyperparameter_tune_model(
    model_type=model_type,
    gauge_ids=basin_ids,
    datamodule_config=datamodule_config,
    training_config=training_config,
    output_dir_study=output_dir,
    experiment_name=experiment_name,
    n_trials=num_trials,
    search_spaces_dir="/Users/cooper/Desktop/hydro-forecasting/experiments/HPT/search-spaces",
)

Seed set to 42
INFO:hydro_forecasting.experiment_utils.hyperparameter_tune_model:Optuna study storage: sqlite:////Users/cooper/Desktop/hydro-forecasting/experiments/HPT/HPT_tide_tajikistan/HPT_tide_tajikistan_study.db, Name: HPT_tide_tajikistan
[I 2025-05-15 13:25:02,020] Using an existing study with name 'HPT_tide_tajikistan' instead of creating a new one.
INFO:hydro_forecasting.experiment_utils.checkpoint_manager:Creating new attempt: /Users/cooper/Desktop/hydro-forecasting/experiments/HPT/HPT_tide_tajikistan/checkpoints/tide/run_6/attempt_0
INFO:hydro_forecasting.experiment_utils.checkpoint_manager:Creating new attempt: /Users/cooper/Desktop/hydro-forecasting/experiments/HPT/HPT_tide_tajikistan/logs/tide/run_6/attempt_0
INFO:hydro_forecasting.experiment_utils.hyperparameter_tune_model:Trial 6 suggested HPs: {'input_length': 100, 'learning_rate': 1.2014712806279581e-05, 'hidden_size': 115, 'dropout': 0.4947194260978343, 'num_encoder_layers': 1, 'num_decoder_layers': 2, 'decoder_outpu